# Codes I think it is not need but as soon as I delete it, it probably will be

## Miscelaneous

In [ ]:
"""
print comparing the values estimated by ppl_scikit and the ones used to generate
the gaussians
"""
n_gaussian = 2
print(np.sqrt(np.abs(covariances[n_gaussian])) * np.sqrt(2))
print(dispersion_x[n_gaussian], dispersion_y[n_gaussian])

In [ ]:
# All gaussians for the ground truth parameters
all_gaussians = list(zip(list(zip(xs, ys)),
                         [np.array([[(wx/(np.sqrt(2)))**2, 0], [0, (wy/(np.sqrt(2)))**2]]) for
                          wx, wy in zip(dispersion_x, dispersion_y)],
                         amplitude))

## YOLO

In [ ]:
# Plots bounding boxes from results of YOLO using cv2
for result in results:
  boxes = result.boxes.cpu().numpy() # get boxes on cpu in numpy
  for box in boxes: # iterate boxes
    r = box.xyxy[0].astype(int) # get corner points as int
    print(r) # print boxes
    cv2.rectangle(img, r[:2], r[2:], (0,255,0), 2) # draw boxes on img
cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Gaussians estimations

In [ ]:
@timer_decorator
def ppl_fit_gaussians(x,
                      y,
                      image,
                      means_init = None,
                      center_coordinates = None,
                      em_iterations = 100,
                      covariance_gaussians_type = 'full',
                      n_gaussians = None,
                      fig_size = (10, 8),
                      plot = True,
                      plot_points = False):
  """
  Just fit the Gaussians using GMM to find means, covariance and amplitude
  : params
  :: x, y - Points that the gaussians are fitted (it doesn't consider z values)
  :: center_coordinates - Initial estimated centers for the Gaussians
  ---
  Notes : The x, y points were select in the project as the values of laplacian
  greater or equal than 0, restricting the coordinates to the points belonging to
  gaussians
  """
  if center_coordinates is not None:
    means_init = np.column_stack((image[0][0, [center_coordinates[:, 1]]].reshape(-1, 1),
                                  image[1][center_coordinates[:, 0], 0].reshape(-1, 1)))

  if means_init is not None:
    n_gaussians = len(means_init)
  gmm, means, covariances = estimate_gaussian_parameters(x, y,
                                                    n_gaussians = n_gaussians,
                                                    means_init = means_init,
                                                    em_iterations = em_iterations,
                                                    covariance_gaussians_type = covariance_gaussians_type)
  if covariance_gaussians_type == 'diag':
    diagonal_matrices = [np.diag(cov) for cov in covariances]
    covariances = diagonal_matrices

  angles = []
  ellipses = []
  for mean, covariance in zip(means, covariances):
      # Generate points on an ellipse around the mean
      eigenvalues, eigenvectors = np.linalg.eigh(covariance)
      angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
      angles.append(angle)
      width, height = 3 * np.sqrt(2 * eigenvalues)  # 2 * sigma
      ellipses.append(patches.Ellipse(xy = mean, width = width, height=height,
                                angle=angle, edgecolor='r', facecolor='none'))

  if plot == True:
    # 2D plot in grayscale - selected image for initial development
    fig, ax = plt.subplots(figsize = fig_size)
    plot_grayscale(image[0], image[1], image[2], ax)
    ax.scatter(means[:,0], means[:,1], s = 5)
    for ellipse in ellipses:
      ax.add_patch(ellipse)
  return means, covariances, angles

In [ ]:
def gmm_predict_gaussians(image,
                          peak_method = 'laplace',
                          covariance_gaussians_types = 'diag',
                          image_ranges = [(0, 0.143), (0, 0.377)],
                          image_size = [143, 377],
                          plot = True,
                          plot_points = False):
  """
  Fits the GMM model in the image depending on the type of covariance chosen
  Used in the pipelines to get the Gaussians estimations
  ---
  : params
  :: image: The image as intensities (the ones generate, not saved ones)
  :: peak_method: which method to find the maximum peaks. 'image' uses the image
  whithout any treatment, 'laplace' uses the transformed laplace image
  """
  nonzero_pairs, z_laplace = laplace_treshold(image[0], image[1], image[2])
  if peak_method == 'laplace':
    coordinates = peak_local_max(-1 * z_laplace, min_distance = 1)
  elif peak_method == 'image':
    coordinates = peak_local_max(image[2], min_distance = 1)
  amplitudes = image[2][coordinates[:, 0], coordinates[:, 1]]
  # Estimation of means, covariances and angles
  (means,
   covariances,
   angles) = ppl_fit_gaussians(nonzero_pairs[:, 0],
                               nonzero_pairs[:, 1],
                               image,
                               means_init = None,
                               em_iterations = 1000,
                               covariance_gaussians_type = covariance_gaussians_types,
                               center_coordinates = coordinates,
                               fig_size = (10, 8),
                               plot = plot)
  if plot_points == True:
    fig, ax = plt.subplots(figsize = (15, 8))
    ax.scatter(nonzero_pairs[:, 0],
               nonzero_pairs[:, 1],
               s = 1)
    aspect_ratio = image_size[1]/image_size[0]
    ax.set_aspect(aspect_ratio)

  # Estimation of amplitude
  amplitudes = find_amplitudes(means,
                               image,
                               image_ranges = image_ranges,
                               image_size = image_size)

  return means, covariances, angles, amplitudes

In [ ]:
# Estimate Gaussian - Final prediction
means, covariances, angles, amplitudes = gmm_predict_gaussians(image,
                                                   peak_method = 'laplace',
                                                   covariance_gaussians_types = 'diag',
                                                   plot = True,
                                                   plot_points = True)

In [ ]:
def yolo_predict_gaussians(model,
                           image_path,
                           image_ranges = [(0, 0.143), (0, 0.377)],
                           image_size = [143, 377]):
  """
  Function using detection task of Yolo.
  Predict bounding boxes of image in "image_path" using "model"
  ---
  : params
  :: model: YOLO model object
  :: image_path: Image location ("~path/image.png")
  :: image_ranges: Ranges of image ([x range, y range])
  :: image_size : Size of image ([x_shape, y_shape])
  """
  # Make the prediction YOLO
  img = cv2.imread(image_path)
  results = model.predict(img)

  # class 0 boxes
  tensor_boxes = results[0].boxes.xywhn
  # Centers
  means = [(float(x * image_ranges[0][-1]),
            float(y * image_ranges[1][-1])) for
            x,y in zip(tensor_boxes[:, 0], tensor_boxes[:, 1])]

  # Covariances
  covariances = [(float(wx * image_ranges[0][-1] / 3),
                  float(wy * image_ranges[1][-1] / 3)) for
                  wx, wy in zip(tensor_boxes[:, 2], tensor_boxes[:, 3])]
  covariances = np.array([np.diag(mat) for mat in covariances])
  covariances = np.square(covariances)

  # Amplitude - needs the intensity from the generated images
  (image,
   amplitude,
   xs,
   ys,
   dispersion_x,
   dispersion_y) = load_image_and_params(n_image,
                                         data_images = mat['F_data'],
                                         data_parameters = mat['Fit_flux'])
  # Defining using the same strategy from GMM
  amplitudes = find_amplitudes(np.array(means),
                               image,
                               image_ranges = image_ranges,
                               image_size = image_size)
  return means, covariances, amplitudes